<div style="border:1px solid black; padding:10px 10px;">
    <strong>CIVIL-321 "Modélisation Numérique des Solides et Structures"</strong><br/><br/>
    <span style="text-decoration:underline;font-weight:bold;">Comment utiliser ce Jupyter Notebook?
    </span><br/><br/>
    Ce <strong>Notebook</strong> est constitué de cellules de texte et de cellule de code. Les cellules de codes doivent être  <strong>executées</strong> pour voir le résultat du programme. Certaines cellules doivent être remplies par vos soins. Pour exécuter une cellule, cliquez dessus simplement et ensuite cliquez sur le bouton "play" (<span style="font: bold 12px/30px Arial, serif;">&#9658;</span>) dans la barre de menu au dessus du notebook. Vous pouvez aussi taper la combinaison de touches <code>shift + enter</code>. Il est important d'éxécuter les cellules de code en respectant leur ordre d'arrivée dans le notebook.
</div>

On vous encourage à poser vos questions et donner votre feedback sur ce notebook sur la plateforme ED Discussion du cours accessible en cliquant sur ce bouton:
 
 
 
<div class="container" >
        <a href="https://edstem.org/eu/courses/409/discussion?category=Exercices">
            <button class="btn btn-primary btn-lg">Ed Discussion</button>
        </a>
</div>

# Série d'exercices : Résolution d'un problème à plusieurs barres

## Introduction

Cette séance d'exercice se fait exclusivement sur Python. Les concepts utilisés dans le Notebook du cours "Barres et treillis" sont censés être maîtrisés. Tout d'abord, importer les modules suivants :

In [ ]:
import numpy as np
from plot import *
import math as m

## Exercice 1 : un exemple complet

### Données :

- L'aire de la barre 0 et 1 : $A = 6\times 10^{-4}$ m$^2$
- Module d'élasticité des barres 0 et 1 : $E = 210\times10^6$ [kN/m$^2$]
- Force au nœud 0 : $P$ = 1000 [kN] (direction -$y$)

In [ ]:
Image(filename = './Figures/exemple.png',  width=200, height=200)

### Objectif

On souhaite connaître le déplacement de chacun des nœuds ainsi que les forces agissant sur ces nœuds dans la direction x et y. Dans ce but, nous détaillerons les étapes de résolution.

#### Étape 1 : Définition des variables du problème

Expliciter :
- La matrice de position des nœuds : _positions = np.array([...])_
- La matrice de connectivité du problème : _conn = np.array([...])_
- La matrice de numérotation d'équation par nœud : _eqn_node = np.array([...])_ 
- La matrice de numérotation d'équation par élément : _eqn_elem = np.array([...])_

In [ ]:
###########
# Solution:
##########


positions = np.array([
    [0, 0],
    [3, 4],
    [0, 4]
])

conn = np.array([
    [0, 1],
    [0, 2]
])

eqn_node = np.array([
    [0, 1],
    [2, 3],
    [4, 5]
])

eqn_elem = np.array([
    [0, 1, 2, 3],
    [0, 1, 4, 5]
])

plot_structure(positions, conn)

- Exprimez la rigidité $k_0$ et $k_1$ des barres 0 et 1 respectivement, à partir des propriétés de la barre.

In [ ]:
###########
# Solution:
##########


E = 210e6 # kN/m^2
A = 6e-4  # m^2

x0 = positions[0, 0]
x1 = positions[1, 0]
x2 = positions[2, 0]

y0 = positions[0, 1]
y1 = positions[1, 1]
y2 = positions[2, 1]

L0 = np.sqrt((x1-x0)**2+(y1-y0)**2)
L1 = np.sqrt((x2-x0)**2+(y2-y0)**2)

k0 = E*A/L0
k1 = E*A/L1

- Définissez le vecteur de rigidité $k_{vec}$ = [$k_0$, $k_1$, etc] contenant les rigidités des barres.

In [ ]:
###########
# Solution:
##########


k_vec = [k0, k1]

- Écrire une fonction calculant la matrice de raideur locale $K_{l}$ d'une barre, en prenant comme entrée la rigidité de la barre $k$. Attention, par rapport à la semaine dernière, nous considérons désormais un espace à deux dimensions, chaque noeud a donc 2 degrés de liberté.

In [ ]:
###########
# Solution:
##########


def CalculerMatriceRigiditeLocale(k):
    
    Kl = k*Matrix([[1, 0, -1, 0],
                   [0, 0, 0, 0],
                   [- 1, 0, 1, 0],
                   [0, 0, 0, 0]])
    
    return(Kl)

- Expliciter cette matrice pour les barres 0 et 1.

In [ ]:
###########
# Solution:
##########


Kl0 = CalculerMatriceRigiditeLocale(k0)
plot_matrix(Kl0, 'Kl0')

Kl1 = CalculerMatriceRigiditeLocale(k1)
plot_matrix(Kl1, 'Kl1')

- Écrire une fonction permettant de calculer la matrice de rotation $\boldsymbol{R}$, connaissant la position des nœuds, la matrice de connectivité ainsi que l'élément e considéré. 

    Pour rappel :
$\boldsymbol{R} = \begin{bmatrix} 
\boldsymbol{r} & 0 \\
0 & \boldsymbol{r}
\end{bmatrix}$ et  $\boldsymbol{r} =\frac{1}{l} \begin{bmatrix} 
x_1 - x_0 & -(y_1-y_0) \\
y_1-y_0 & x_1 - x_0
\end{bmatrix}
$
où $l = \sqrt{(x_1 - x_0)^2 + (y_1 - y_0)^2}.$

In [ ]:
###########
# Solution:
##########


def CalculerMatriceRotation(e, positions, conn):
    
    n_1 = positions[conn[e, 0], :]
    n_2 = positions[conn[e, 1], :]
    
    barre = n_2 - n_1
    
    l = np.linalg.norm(barre)
    
    r = np.array([
        [barre[0]/l, -barre[1]/l],
        [barre[1]/l, barre[0]/l]
    ])
    
    R = np.zeros((4,4))
    R[0:2, 0:2] = r
    R[2:, 2:] = r
        
    return R

- Vérifier l'exactitude de votre fonction en l'appliquant pour la barre 0 et 1.

In [ ]:
###########
# Solution:
##########


R0 = CalculerMatriceRotation(0, positions, conn)
plot_matrix(R0, 'R0')

R1 = CalculerMatriceRotation(1, positions, conn)
plot_matrix(R1, 'R1')

#### Étape 3 : Assemblage de la matrice de rigidité

- Écrire une fonction permettant d'assembler les matrices de rigidités locales dans la matrice de rigidité globale en prenant en compte: 
    - la rigidité des barres (k_vec)
    - la positions des nœuds, la matrice de connectivité (conn)
    - le nombre d'élément (nb_elem)
    - le nombre de degrés de liberté total (nb_ddl) 
    - la matrice de numérotation par élément (eqn_elem).

In [ ]:
#SolutionToRemove

def AssemblerMatriceRigidite(k_vec, positions, conn, nb_elem, nb_ddl_tot, eqn_elem):
    K = np.zeros((nb_ddl_tot, nb_ddl_tot))
    
    for e in range(nb_elem):
        # Construction de la matrice de rigidité locale
        Kl = calculerMatriceRigiditeLocale(k_vec[e])
        # Construction de la matrice de rotation
        R = CalculerMatriceRotation(e, positions, conn)
        # Rotation de la matrice dans le système global de coordonnée
        Klrot = R@Kl@R.T
        
        # Assemblage dans la matrice de rigidité du système à l'aide des
        idx = eqn_elem[e, :]
        for i, global_i in enumerate(idx):
            for j, global_j in enumerate(idx):
                K[global_i, global_j] += Klrot[i, j]
    return K

# entrer nb_elem et nb_ddl pour ce problème
nb_elem = 2
nb_ddl_tot = 6

K = AssemblerMatriceRigidite(k_vec, positions, conn, nb_elem, nb_ddl_tot, eqn_elem)

plot_matrix(K, 'K')

- Vérifier que la position des zéros dans la matrice $\boldsymbol{K}$ est cohérente avec les ddl.

#### Étape 4 : Conditions aux limites et calcul des réactions

- Définir les conditions aux limites (6 conditions) en force et déplacement.

---

 **Solution:**

 ---



- $d_{0x} = 0$
- $f_{0y} = -P$
- $d_{1x} = d_{1y} = 0$
- $d_{2x} = d_{2y} = 0$

- Calculer le déplacement de la composante restante (non nulle).

In [ ]:
###########
# Solution:
##########


P = 1000

d0y = -P/K[1, 1]
print(f'd0y = {d0y} m')

- Calculer les réactions $\lbrace f \rbrace$ grâce à la formule $\lbrace f \rbrace$ = $[K] \lbrace d \rbrace$.

In [ ]:
###########
# Solution:
##########


d = np.array([
    [0],
    [d0y],
    [0],
    [0],
    [0],
    [0]
])

f = np.dot(K, d)
plot_matrix(f, 'f')

## Exercice 2 : Assemblage d'un système de ressorts

Lors de l’exercice 2 de la série 1, nous avons vu comment résoudre numériquement un système de ressorts assemblés en série. L’objectif de cet exercice est de standardiser le code afin de pouvoir assembler n’importe quelle configuration de ressorts. Cette standardisation est réalisée en définissant une matrice de connectivité. Le rôle de la matrice de connectivité est de lier la numérotation globale des nœuds (sur la structure entière) à la numérotation locale (sur un seul élément).

1. Considérons la numérotation locale de la figure suivante correspondant à un ressort de rigidité $k^e$ à deux degrés de liberté $u^e_1$ et $u^e_2$. Écrivez une fonction calculant la matrice de rigidité locale $\boldsymbol{K^l_{ij}}$ (2x2) pour cet élément ressort.

In [ ]:
Image(filename = './Figures/assemblage_ressort.png',  width=300, height=300)

In [ ]:
###########
# Solution:
##########


def CalculerMatriceRigiditeLocale_ex2(k):
    
    Kl = k*Matrix([[1, -1],
                   [-1, 1]])
    return(Kl)

2. Nous allons à présent modéliser le système de ressorts suivant composé de 4 éléments et de 5 nœuds. Exprimez la matrice de connectivité $\boldsymbol{C}$ (4x2) du système de la figure suivante :

In [ ]:
Image(filename = './Figures/systeme_ressort.png',  width=500, height=500)

In [ ]:
###########
# Solution:
##########


conn = np.array([
    [1, 0],
    [3, 1],
    [1, 2],
    [2, 4]
])

3. La matrice de connectivité permet ainsi de standardiser l'assemblage de matrices de rigidité locales $\boldsymbol{K^l}$ dans la matrice de rigidité du système $\boldsymbol{K}$. La fonction d'assemblage a été définie dans l'exercice précédemment. Une simplification de cette fonction dans le cas d'éléments non inclinés est telle que :

In [ ]:
def AssemblerMatriceRigidite_ex2(ki, nb_elem, nb_ddl, conn):
    K = np.zeros((nb_ddl, nb_ddl))
    for e in range(nb_elem):
        Kl = CalculerMatriceRigiditeLocale_ex2(ki)
        idx = conn[e, :]
        for i, gi in enumerate(idx):
            for j, gj in enumerate(idx):
                K[gi, gj] += Kl[i, j]
    return K

Vérifiez que la matrice de rigidité assemblée correspondent à celle prédite analytiquement lors de l’exercice 3.4 de la série 1 avec $k_i$=4.

In [ ]:
###########
# Solution:
##########


# entrer nb_elem et nb_ddl pour ce problème ainsi que la rigidité de chaque ressort ki
nb_elem = 4
nb_ddl_tot = 5
ki = 4

K = AssemblerMatriceRigidite_ex2(ki, nb_elem, nb_ddl_tot, conn)
plot_matrix(K, 'K')

4. Appliquez les conditions limites du système $u_4$ = $u_5$ = 0 et résolvez numériquement le système pour $k_i$ = 4, $F_1$ = 12, $F_2$ = 0 et $F_3$ = −9.

In [ ]:
###########
# Solution:
##########


# Application des conditions aux limites
    # En force :
F = np.zeros((nb_ddl_tot, 1))

F[0] = 12
F[1] = 0
F[2] = -9

    # En déplacement :
u3 = 0 # ddl 3
u4 = 0 # ddl 4

    # ddl libres :
ddl_libres = [i for i in range(0, nb_ddl_tot)]
ddl_libres.remove(3)
ddl_libres.remove(4)

# Résolution du système
K_libre = K[ddl_libres, :][:, ddl_libres]
F_libre = F[ddl_libres]
u_libre = np.linalg.solve(K_libre, F_libre)

# Solution 
u = np.zeros((nb_ddl_tot, 1))
u[ddl_libres] = u_libre
plot_matrix(u, 'u')

F = K@u
plot_matrix(F, 'F')

## Exercice 3 : Modélisation d'une structure en treillis

Pour cette exercice, nous souhaitons étudier la déformation d’un pont ferroviaire en treillis à l’aide du code développé dans l’exercice 1.

In [ ]:
Image(filename = './Figures/exercice1.png',  width=400, height=400)

avec :
   - $E$ = 210 GPa 
   - $A_1$ = 20 000 $mm^2$ 
   - $A_2$ = 10 000 $mm^2$ 
   - $f$ = 100 kN

On donne la matrice de connectivité et les positions des nœuds définis comme suit :

In [ ]:
positions = np.array([
    [0, 0],
    [4, 0],
    [8, 0],
    [12, 0],
    [16, 0],
    [20, 0],
    [24, 0],
    [4, 6],
    [8, 6],
    [16, 6],
    [20, 6]
])

conn = np.array([
    [0, 1],
    [1, 2],
    [2, 3],
    [3, 4],
    [4, 5],
    [5, 6],
    [0, 7],
    [7, 8],
    [8, 9],
    [9, 10],
    [10, 6],
    [1, 7],
    [2, 8],
    [4, 9],
    [5, 10],
    [2, 7],
    [3, 8],
    [3, 9],
    [4, 10],
])

1. Cette structure est discrétisée à l’aide d’éléments barres dont la rigidité hors plan est supposée nulle. Écrire l’expression de la matrice de rigidité locale pour l’élément barre présenté à la figure suivante :

In [ ]:
Image(filename = './Figures/exercice1_q1.png',  width=300, height=300)

---

 **Solution:**

 ---



Dans une barre, les déplacements hors plan n’ont aucune rigidité associée et la matrice de rigidité
s’écrit donc : 
    
$$\begin{equation} \boldsymbol{K^l} = \frac{EA}{l}
   \left[\begin{array}{cccc}
     1  & 0  & -1  & 0  \\
     0  & 0  &  0  & 0 \\
    -1  & 0  &  1  & 0  \\
     0  & 0  &  0  & 0  
   \end{array}\right].
   \end{equation}$$

2. Construisez une fonction calculant la matrice de numérotation par élément $E_{qe}$ à partir de la matrice de connectivité prenannt en entrée :
    - la matrice de connectivité ;
    - le nombre d'éléments ;
    - le nombre de ddl ;
    - le nombre de nœuds par élément ;
    - le nombre de ddl par nœuds.
    
Vérifiez le résultat obtenu.

In [ ]:
###########
# Solution:
##########


def precalculerNumeroEquation(conn, nb_elem, nb_ddl, nb_noeuds_p_elem, nb_ddl_p_noeuds):
    eqn_elem = np.zeros((nb_elem, nb_ddl), dtype=int)
    for e in range(nb_elem):
        for n in range(nb_noeuds_p_elem):
            for d in range(nb_ddl_p_noeuds):
                eqn_elem[e, n*nb_ddl_p_noeuds+d]=conn[e,n]*nb_ddl_p_noeuds+d
    return eqn_elem

# application

nb_elem = len(conn)
nb_nodes = len(positions)
nb_ddl = 4
nb_noeuds_p_elem = 2
nb_ddl_p_noeuds = 2

eqn_elem = precalculerNumeroEquation(conn, nb_elem, nb_ddl, nb_noeuds_p_elem, nb_ddl_p_noeuds)
plot_matrix(eqn_elem, 'Eqe')

3. Une fois la matrice de raideur $K$ définie grâce à la fonction _AssemblerMatriceRigidite_, il est possible d'afficher son profil avec le commande :

    _mh.ColoredMatrix(K).profile(remove_zeros=True)_
   
Étant donné que nous nous intéressons pas aux valeurs exactes des coefficients, nous fixons la rigidité de chaque barre arbitrairement à 1. Commenter le profil obtenu.

In [ ]:
###########
# Solution:
##########


nb_elem = len(conn)

nb_nodes = len(positions)
nb_ddl_tot = 2*nb_nodes

E = 210*10**9

k_vec = np.zeros(nb_elem)
for i in range(nb_elem):
    k_vec[i] = 1

K = AssemblerMatriceRigidite(k_vec, positions, conn, nb_elem, nb_ddl_tot, eqn_elem)
mh.ColoredMatrix(K).profile(remove_zeros=True)

---

 **Solution:**

 ---



Dans la plupart des modèles éléments finis, les éléments sont uniquement connectés à leur voisins géométriques proches et la matrice de rigidité est donc éparse (peu de coefficients non-nuls).

4. Appliquez les conditions d'appuis et calculez les déplacements ainsi que les forces en chacun des nœuds.

In [ ]:
###########
# Solution:
##########


# Données :
A1 = 20*10**(-3)
A2 = 10*10**(-3)
E = 210*10**9
f = 100*10**3

# Vecteur de rigidité 
k_vec = []
for e in range(0, nb_elem):
    n_1 = positions[conn[e, 0], :]
    n_2 = positions[conn[e, 1], :]
    barre = n_2 - n_1
    l = np.linalg.norm(barre)
    
    if e < 11 :
        k_vec.append(E*A1/l)
    else :
        k_vec.append(E*A2/l)

# Matrice de raideur
K = AssemblerMatriceRigidite(k_vec, positions, conn, nb_elem, nb_ddl_tot, eqn_elem)

# Application des conditions aux limites
    # En force :
F = np.zeros((nb_ddl_tot, 1))

nb_ddl_p_noeuds = 2
F[1*nb_ddl_p_noeuds+1] = -f    
F[2*nb_ddl_p_noeuds+1] = -f    
F[3*nb_ddl_p_noeuds+1] = -f    
F[4*nb_ddl_p_noeuds+1] = -f    
F[5*nb_ddl_p_noeuds+1] = -f   

    # En déplacement :
u0x = 0 # ddl 0
u0y = 0 # ddl 1
u6y = 0 # ddl 13

    # ddl libres :
ddl_libres = [i for i in range(0, nb_ddl_tot)]
ddl_libres.remove(0)
ddl_libres.remove(1)
ddl_libres.remove(13)

# Résolution du système
K_libre = K[ddl_libres, :][:, ddl_libres]
F_libre = F[ddl_libres]
u_libre = np.linalg.solve(K_libre, F_libre)

# Solution 
u = np.zeros((nb_ddl_tot, 1))
u[ddl_libres] = u_libre
plot_matrix(u, 'u')

F = K@u
plot_matrix(F, 'F')

In [ ]:
###########
# Solution:
##########


# Déformation de la structure amplifiée 500 fois.

plt.rcParams['figure.figsize'] = [10, 10]
fig, ax = plt.subplots()
ret = plot_structure(positions, conn, show_elem_indexes=False, show_nodes=False, linestyle='--', ax=ax)
ret = plot_structure(positions+500*u.reshape(11, 2), conn, show_elem_indexes=False, ax=ax)